# Lab Session: Building a Recommender System with LightFM

## Step 0: Load Libraries and Data

In [1]:
# Standard library (ex. pathlib, math, os, sys)
from pathlib import Path

# Third-party libraries (ex. numpy, pandas, matplotlib, scikit-learn)
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Définition des chemins locaux
local_articles_path = Path("../data/raw/articles.csv").resolve()
local_customers_path = Path("../data/raw/customers.csv").resolve()
local_transactions_path = Path("../data/raw/transactions_train.csv").resolve()

# URLs distantes (GitHub)
# Récupérer les données depuis Kaggle (attention aux droits d'accès)
articles_url = (
    "https://media.githubusercontent.com/media/auduvignac/"
    "..."
)
customers_url = (
    "https://media.githubusercontent.com/media/auduvignac/"
    "..."
)
transactions_url = (
    "https://media.githubusercontent.com/media/auduvignac/"
    "..."
)


def load_dataset(local_path, url, name):
    """
    Charge un fichier CSV depuis un chemin local, ou le télécharge depuis
    une URL si le fichier n'existe pas localement.

    Le fichier est ensuite sauvegardé localement pour les exécutions
    futures (fallback).

    Parameters
    ----------
    local_path : pathlib.Path
        Chemin vers le fichier local.
    url : str
        URL distante du fichier CSV.
    name : str
        Nom affiché lors du chargement (utilisé pour les logs).

    Returns
    -------
    pandas.DataFrame
        Le contenu du fichier CSV sous forme de DataFrame.
    """
    if local_path.exists():
        print(f"{name} chargé depuis : {local_path}")
        return pd.read_csv(local_path)
    else:
        print(f"{name} non trouvé localement, téléchargement depuis l'URL...")
        df = pd.read_csv(url)
        local_path.parent.mkdir(parents=True, exist_ok=True)
        df.to_csv(local_path, index=False)
        print(
            f"{name} téléchargé et enregistré localement dans : {local_path}"
        )
        return df


# Chargement des jeux de données
try:
    print("Chargement des données...")
    articles = load_dataset(local_articles_path, articles_url, "articles")
    customers = load_dataset(local_customers_path, customers_url, "customers")
    transactions = load_dataset(local_transactions_path, transactions_url, "transactions")
    print("Données chargées avec succès.\n")

except Exception as e:
    print("Échec du chargement des données.")
    print("Vérifiez la validité des URL ou la connexion internet.")
    print(e)

Chargement des données...
articles chargé depuis : /home/aurelien/workspace/building-recommender-with-lightfm/data/raw/articles.csv
customers chargé depuis : /home/aurelien/workspace/building-recommender-with-lightfm/data/raw/customers.csv
transactions chargé depuis : /home/aurelien/workspace/building-recommender-with-lightfm/data/raw/transactions_train.csv
Données chargées avec succès.



## Step 1: Data Exploration & Understanding

**Objective: Get familiar with the H&M dataset structure and characteristics.**

Key Questions to Explore:

- What does the interaction data look like ? How many unique users and items do we have ?

In [3]:
# What does the interaction data look like ?
print("Aperçu des données :")
print("Articles :")
print(articles.head())
print(f"Articles : {articles.shape[0]} lignes, {articles.shape[1]} colonnes")
print("Custommers :")
print(customers.head())
print(
    f"Custommers : {customers.shape[0]} lignes, {customers.shape[1]} colonnes"
)
print("Transactions :")
print(transactions.head())
print(f"Transactions : {transactions.shape[0]} lignes, {transactions.shape[1]} colonnes")
# How many unique users and items do we have ?
unique_users = transactions['customer_id'].nunique()
unique_items = transactions['article_id'].nunique()
print(f"Utilisateurs uniques : {unique_users}")
print(f"Articles uniques : {unique_items}")

Aperçu des données :
Articles :
   article_id  product_code          prod_name  product_type_no  \
0   108775015        108775          Strap top              253   
1   108775044        108775          Strap top              253   
2   108775051        108775      Strap top (1)              253   
3   110065001        110065  OP T-shirt (Idro)              306   
4   110065002        110065  OP T-shirt (Idro)              306   

  product_type_name  product_group_name  graphical_appearance_no  \
0          Vest top  Garment Upper body                  1010016   
1          Vest top  Garment Upper body                  1010016   
2          Vest top  Garment Upper body                  1010017   
3               Bra           Underwear                  1010016   
4               Bra           Underwear                  1010016   

  graphical_appearance_name  colour_group_code colour_group_name  ...  \
0                     Solid                  9             Black  ...   
1         

- What's the sparsity of the dataset ? (Compare total possible interactions vs actual interactions) ?

**interaction definition:** In the context of recommender systems, an interaction typically refers to a user-item pair where a user has shown interest in or engaged with an item. This can include actions like viewing, purchasing, or rating an item.

**Sparsity:** Sparsity can be calculated by taking the ratio of zeros in a dataset to the total number of elements. Addressing sparsity will affect the accuracy of your machine-learning model.

In [ ]:
# What's the sparsity of the dataset ?
# interaction : refers to a user-item pair where a user has shown interest in or engaged with an item.
# customers x articles
nb_total_interactions = unique_users * unique_items
print(f"Total number of possible interactions : {nb_total_interactions}")
nb_interactions = transactions.shape[0]
print(f"interactions : {nb_interactions}")
sparsity = 1 - (nb_interactions / nb_total_interactions)
print(f"Sparsité du dataset : {sparsity:.4f} ({sparsity * 100:.2f}%)")

- How are interactions distributed across users and items ? Are there power users or blockbuster items ?

In [ ]:
# Number of power users and blockbuster items
number_of_power_users = 10
number_of_blockbuster_items = 10
# power users
print("Power users :")
top_users = (
    transactions["customer_id"].value_counts().head(number_of_power_users)
)
print(top_users)
# blockbuster items
print("Blockbuster items :")
top_items = (
    transactions["article_id"].value_counts().head(number_of_blockbuster_items)
)
print(top_items)

En l'occurrence les utilisateurs peuvent êtres des entreprises qui réalisent un grand nombre d'achats. Il s'agit de différencier les entreprises des utilisateurs lambda. Il faut donc travailler avec des intervalles.
Un autre point d'intêret est le nombre de transactions par catégories de produits.
Il faut également prendre garde à conserver une diversité des catégories de produits dans les recommandations.
Nombre de catégories achetées par un utilisateur. Peut-être chaque utilisateur se concentre sur un nombre restreint de catégories de produits.
Concernant la recommandation, il est tout à fait envisageable de recommander des produits même s'il ne sont pas de la bonne couleur (i.e. plutôt que de recommander des articles, on peut recommander des catégories de produits).

- What time period does the data cover ? Are there seasonal patterns ?

In [ ]:
# seasonal patters
# Spring : 20/03 - 20/06
# Summer : 21/06 - 22/09
# Autumn : 23/09 - 20/12
# Winter : 21/12 - 19/03
# Convert 't_dat' to datetime
transactions['t_dat'] = pd.to_datetime(transactions['t_dat'], format='%Y-%m-%d')
# Extract month and day
transactions['month'] = transactions['t_dat'].dt.month
transactions['day'] = transactions['t_dat'].dt.day
# Define seasons based on month and day
def get_season(row):
    if (row['month'] == 3 and row['day'] >= 20) or (row['month'] in [4, 5]) or (row['month'] == 6 and row['day'] < 21):
        return 'Spring'
    elif (row['month'] == 6 and row['day'] >= 21) or (row['month'] in [7, 8]) or (row['month'] == 9 and row['day'] < 23):
        return 'Summer'
    elif (row['month'] == 9 and row['day'] >= 23) or (row['month'] in [10, 11]) or (row['month'] == 12 and row['day'] < 21):
        return 'Autumn'
    else:
        return 'Winter'
transactions['season'] = transactions.apply(get_season, axis=1)
# Count interactions per season
season_counts = transactions['season'].value_counts()
print("Interactions par saison :")
print(season_counts)
# Visualize seasonal patterns
season_counts.plot(kind='bar', color='skyblue')
plt.title('Interactions par saison')
plt.xlabel('Saison')
plt.ylabel('Nombre d\'interactions')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

- What metadata is available for items and customers ?